### MLR Model

In [22]:
from sklearn.linear_model import LinearRegression

In [23]:
from sklearn.linear_model import LinearRegression
def MLR_Model(X_train,y_train):
    mlr = LinearRegression()
    mlr.fit(X_train,y_train)
    return mlr

### Function to make ANN model

initial_weights and bias will be from MLR

In [66]:
def ANN_Model (initial_weights,bias): 
    model = Sequential()
    model.add(Input(shape = 4,))
    model.add(Dense(1, activation = 'relu', name = 'hidden'))
    model.add(Dense(1,activation = 'linear'))


    #initialize weights and bias in hidden layer
    hidden_layer = model.get_layer('hidden')
    #layer.set_weights([weights_array, bias_array])
    hidden_layer.set_weights([np.array([initial_weights]).reshape((4, 1)), np.array([bias])])


    #compile 
    model.compile(loss='mean_absolute_error',optimizer=tf.keras.optimizers.Adam(learning_rate=0.1))
    return model



### 5-fold MLR -ANN with RMSE and R-square (whole dataset)

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from numpy import loadtxt
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Input, Dense
from tensorflow.keras import layers
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

In [3]:
import time

In [4]:
from keras import metrics

In [5]:
whole_data = pd.read_csv(r"D:\Thesis\Datasets\Final Yield_Weather (Mindanao).csv") 

In [6]:
whole_data.head()

,Unnamed: 0,Year,Location,Wind_Direction,Wind_Speed_(km/h),temperature_2m (°C),relativehumidity_2m (%),dewpoint_2m (°C),surface_pressure (hPa),et0_fao_evapotranspiration (mm),...,soil_moisture_28_to_100cm (m³/m³),direct_radiation (W/m²),diffuse_radiation (W/m²),rain (mm),weathercode (wmo code),windgusts_10m (km/h),cloudcover (%),Production (mt),Area (ha),Yield (mt/ha)
0,0,1990,AGUSAN DEL NORTE,NW,4.1,25.15,78.36,20.88,975.30,0.16,...,0.28,140.02,70.89,1701.0,1,9.4,100,226.68,200.0,1.133400
1,1,1991,AGUSAN DEL NORTE,SE,3.1,25.06,77.99,20.73,975.55,0.16,...,0.27,138.79,73.80,1634.2,1,11.5,100,231.68,204.0,1.135686
2,2,1992,AGUSAN DEL NORTE,SE,3.1,25.39,74.29,20.11,975.90,0.17,...,0.21,144.62,73.53,1433.2,1,7.9,100,219.08,199.0,1.100905
3,3,1993,AGUSAN DEL NORTE,SE,3.6,24.92,79.60,20.94,975.66,0.15,...,0.28,136.18,72.02,2017.8,1,8.3,100,231.47,203.0,1.140246
4,4,1994,AGUSAN DEL NORTE,NW,3.6,24.96,79.45,20.96,975.32,0.15,...,0.29,137.68,71.24,1722.0,1,8.6,100,238.09,208.0,1.144663


In [8]:
#define target and predictors 

X = whole_data[['weathercode (wmo code)', 'temperature_2m (°C)',
       'relativehumidity_2m (%)', 'dewpoint_2m (°C)', 'rain (mm)',
       'surface_pressure (hPa)', 'cloudcover (%)',
       'et0_fao_evapotranspiration (mm)', 'vapor_pressure_deficit (kPa)',
       'Wind_Direction', 'Wind_Speed_(km/h)', 'windgusts_10m (km/h)',
       'soil_temperature_28_to_100cm (°C)',
       'soil_moisture_28_to_100cm (m³/m³)', 'direct_radiation (W/m²)',
       'diffuse_radiation (W/m²)']]
y = whole_data['Yield (mt/ha)']

In [9]:
#function for rmse
from keras import backend
 
def rmse(y_true, y_pred):
 return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))

In [10]:
from sklearn.metrics import r2_score

In [11]:
from sklearn.linear_model import LinearRegression

In [12]:
from sklearn.linear_model import LinearRegression
def MLR_Model(X_train,y_train):
    mlr = LinearRegression()
    mlr.fit(X_train,y_train)
    return mlr

In [13]:
def ANN_Model (initial_weights,bias): 
    model = Sequential()
    model.add(Input(shape = 16,)) # number of input features 
    model.add(Dense(1, activation = 'relu', name = 'hidden')) # one neuron in hidden layer, relu activation
    model.add(Dense(1,activation = 'linear')) # one neuron in output layer, linear activation


    #initialize weights and bias in hidden layer
    hidden_layer = model.get_layer('hidden')
    #layer.set_weights([weights_array, bias_array])
    hidden_layer.set_weights([np.array([initial_weights]).reshape((16, 1)), np.array([bias])])


    #compile 
    model.compile(loss='mean_absolute_error',optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),metrics = [metrics.mean_absolute_error,rmse])
    return model



In [ ]:
start_time = time.time()

#initializing KFold object
kfold = KFold(n_splits=10,shuffle=True, random_state=42)
mae_scores = []
r2_scores = []

for train_index, test_index in kfold.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]  # Predictors for train & test
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]  # Target for train, test

    # scaling fetaure sets 
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    mlr_model = MLR_Model(X_train,y_train) #perform MLR
    ann_model = ANN_Model(mlr_model.coef_,mlr_model.intercept_) #Intercept and coeff of MLR will be initailzied as ANN weights
    ann_model.fit(X_train, y_train, epochs=500, batch_size=32, verbose=0) #fit ann model to data
    y_pred = ann_model.predict(X_test) #use ann model to predict test set
    mae = mean_absolute_error(y_test, y_pred) #measure mean absolute value
    print(f"MAE: {mae}")
    r2 = r2_score(y_test,y_pred) #measure r-squared
    print(f"R2: {r2}")
    print(f"Intercept: {mlr_model.intercept_}")
    print(f"Coefficients: {list(zip(mlr_model.coef_))}")

end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")